In [1]:
import argparse
import gzip, os, csv
import numpy as np
import random
import time

# word2vec learning

In [2]:
!pip install gensim --user

     |████████████████████████████████| 57 kB 4.0 MB/s 
     |████████████████████████████████| 11.2 MB 15.5 MB/s 
     |████████████████████████████████| 204 kB 63.3 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
     |████████████████████████████████| 895 kB 54.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149867 sha256=29965e7e2ffa62187eb8faf54dcbe2c3672b87d7d0af44c0ffe1e9ac4dbacd2e
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
  Created wheel for python-louvain: filename=python_louvain-0.15-py3-none-any.whl size=9413 sha256=27cd98086b3775c6e06763240e4411a1259a0d5e26b45b206fe787d45e79d7c0
  Stored in directory: /root/.cache/pip/wheels/c7/4c/ef/427db8777620f7fa12a6404f1527a39536a812f46a4d1044d3
Successfully built python-Levenshtein python-louvain
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-w

In [2]:
import gensim
class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            #print (fname)
            for line in open(os.path.join(self.dirname, fname)):
                yield line.lower().split()

In [3]:
!wget https://github.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/raw/master/resources/textcorpus/people-wikipedia.zip 


--2022-02-20 22:40:22--  https://github.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/raw/master/resources/textcorpus/people-wikipedia.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/master/resources/textcorpus/people-wikipedia.zip [following]
--2022-02-20 22:40:22--  https://raw.githubusercontent.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/master/resources/textcorpus/people-wikipedia.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 208786 (204K) [application/zip]
Saving to: ‘people-wikipedia.zip’

people-wikipedia.zi 100%[===============

In [4]:
!unzip people-wikipedia.zip

Archive:  people-wikipedia.zip
  inflating: people/BarackObama.txt  
  inflating: people/DonaldTrump.txt  
  inflating: people/BradPitt.txt     
  inflating: people/AngelinaJolie.txt  


In [5]:
sentences = MySentences('people/') # a memory-friendly iterator

In [6]:
model = gensim.models.Word2Vec(size=200, workers=8, window=5, sg=1, negative=15, iter=5)
model.build_vocab(sentences)


In [7]:
corpus_count = model.corpus_count
model.train(sentences, total_examples=corpus_count, epochs =5)

(253909, 445045)

In [8]:
model.wv.most_similar('casino')

[('often', 0.9985657930374146),
 ('hotel', 0.9982964992523193),
 ('saying,', 0.9975336194038391),
 ('nearly', 0.9972118139266968),
 ('show', 0.9971989989280701),
 ('fred', 0.9970407485961914),
 ('reported', 0.9969844818115234),
 ('foundation,', 0.9968953132629395),
 ('figure', 0.9967403411865234),
 ('threatened', 0.9966793060302734)]

In [9]:
import gensim.downloader as api

In [10]:
word2vec_model = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [15]:
# King — Man + Woman = Queen 
word2vec_model.most_similar(positive=['king','woman'], negative=['man'],topn=5) 


[('queen', 0.7118192911148071),
 ('monarch', 0.6189674139022827),
 ('princess', 0.5902431011199951),
 ('crown_prince', 0.5499460697174072),
 ('prince', 0.5377321243286133)]

In [20]:
# Canada — Netherlands + Amsterdam = ? 
word2vec_model.most_similar(positive=['Canada','Amsterdam'], negative=['Netherlands'],topn=5) 


[('Toronto', 0.604716956615448),
 ('Winnipeg', 0.5867977142333984),
 ('Montreal', 0.5769765377044678),
 ('Vancouver', 0.566444993019104),
 ('Ontario', 0.5597741603851318)]

# RDF2Vec training

In [21]:
!wget https://github.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/raw/master/resources/rdf/human_kg_train.nt

--2022-02-20 22:53:28--  https://github.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/raw/master/resources/rdf/human_kg_train.nt
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/master/resources/rdf/human_kg_train.nt [following]
--2022-02-20 22:53:29--  https://raw.githubusercontent.com/MaastrichtU-IDS/UM_KEN4256_KnowledgeGraphs/master/resources/rdf/human_kg_train.nt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3790369 (3.6M) [text/plain]
Saving to: ‘human_kg_train.nt’

human_kg_train.nt   100%[===================>]   3.61M  --.-KB/s    in 0.04s 

In [28]:
!git clone https://github.com/MaastrichtU-IDS/pyRDF2Vec
#!pip install rdflib
#!pip install pyRDF2vec --user

Cloning into 'pyRDF2Vec'...
remote: Enumerating objects: 650, done.
remote: Total 650 (delta 0), reused 0 (delta 0), pack-reused 650
Receiving objects: 100% (650/650), 2.74 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (350/350), done.


In [29]:
import os
os.chdir('pyRDF2Vec')

In [31]:
!pip install -r requirements.txt 

     |████████████████████████████████| 281.4 MB 43 kB/s 
     |████████████████████████████████| 198 kB 63.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=845821f597f4fb44c6ce8e9bf37993a7ee4ede9a9f775fee5c9163381bf6e5a8
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [33]:
from rdf2vec.converters import rdflib_to_kg
from rdf2vec.walkers import RandomWalker
from rdf2vec import RDF2VecTransformer

In [34]:
rdf_file = "human_kg_train.nt"
kg = rdflib_to_kg(rdf_file, filetype='ntriples')

100%|██████████| 30195/30195 [00:01<00:00, 21528.93it/s]


In [37]:

# define a walker to extract a maximum of 5 walks of depth 4 for each entity using the Random walking strategy and Uniform sampling strategy
walkers = [RandomWalker(4, 100)]

# Create embeddings with random walks
transformer = RDF2VecTransformer(walkers= walkers, sg=1)

<SparkContext master=local[10] appName=pyspark-shell>


In [38]:
# Get entities from the KG
all_entities = kg.get_all_entities()


In [39]:
all_entities[:10]

['Wrocław',
 '""1886-07-11T00:00:00Z""^^http://www.w3.org/2001/XMLSchema#dateTime',
 'Josceline Percy',
 'http://www.wikidata.org/entity/Q14915627',
 'http://www.wikidata.org/entity/Q166646',
 'http://www.wikidata.org/entity/Q21537613',
 'Celina Szymanowska',
 'http://www.wikidata.org/entity/Q49261',
 'http://www.wikidata.org/entity/Q336497',
 'http://www.wikidata.org/entity/Q28053544']

In [40]:

# Entities should be a list of URIs that can be found in the Knowledge Graph
walk_embeddings = transformer.fit_transform(kg, all_entities)

./walks/randwalks_n100_depth4_pagerank_uniform.txt
Time elapsed to generate features: 00:00:24
Extracted 0 walks for 19238 instances!
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform


In [45]:
transformer.model_.wv.most_similar('http://www.wikidata.org/entity/Q1080992')  

[('http://www.wikidata.org/entity/Q1808753', 0.9936909675598145),
 ('http://www.wikidata.org/entity/Q1851153', 0.975959300994873),
 ('http://www.wikidata.org/entity/Q2034816', 0.9738614559173584),
 ('http://www.wikidata.org/entity/Q2014562', 0.9724259376525879),
 ("b'\\xe0\\xd4\\x12\\xba\\x82d\\xe6\\xb0'", 0.9703022241592407),
 ('http://www.wikidata.org/entity/Q1039034', 0.9692423343658447),
 ('http://www.wikidata.org/entity/Q6832867', 0.9675869941711426),
 ('http://www.wikidata.org/entity/Q527177', 0.9666233658790588),
 ('http://www.wikidata.org/entity/Q3889237', 0.9658535122871399),
 ('http://www.wikidata.org/entity/Q11066700', 0.9657449722290039)]

In [48]:
#Remember: King — Man + Woman = Queen  (for gender relationship)
#Remember: Queen - Woman + Man = King
# we know that Q1080992 hasMother Q3916314 
#  what is the mother of Q80158556   ?
# vector(Q1080992) - vector(Q3916314) = vector(Q80158556) - vector(Q47522651)
# vector(Q47522651) =   vector(Q3916314) + vector(Q80158556) - vector(Q1080992)
# Q80158556 + Q1080992 - Q3916314 
transformer.model_.wv.most_similar(positive=['http://www.wikidata.org/entity/Q80158556','http://www.wikidata.org/entity/Q3916314'], 
                                            negative=['http://www.wikidata.org/entity/Q1080992'], topn=10)

[("b'\\xc3<\\x99\\x8eN\\xffo{'", 0.9523777961730957),
 ('http://www.wikidata.org/entity/Q75328392', 0.9489349126815796),
 ('http://www.wikidata.org/entity/Q75601713', 0.9489009380340576),
 ('http://www.wikidata.org/entity/Q26208330', 0.9485467672348022),
 ('http://www.wikidata.org/entity/Q77981274', 0.9485460519790649),
 ('http://www.wikidata.org/entity/Q1971996', 0.9470655918121338),
 ('http://www.wikidata.org/entity/Q3150', 0.9465994834899902),
 ('http://www.wikidata.org/entity/Q22662561', 0.9462426900863647),
 ('http://www.wikidata.org/entity/Q16869843', 0.9457598924636841),
 ('http://www.wikidata.org/entity/Q12677896', 0.9454984664916992)]

In [49]:
transformer.model_.wv.most_similar('http://www.wikidata.org/entity/Q47522651') 

[("b'\\xd6i\\x9c\\x14\\xebfE0'", 0.999653160572052),
 ("b'\\xb7\\x87\\xe1-t\\x88\\xd5='", 0.9996531009674072),
 ('b"\\xd7\'\\x8b\\xec`\\xa1\\x04\\x13"', 0.9996498227119446),
 ("b'\\x014\\xb1\\xaa\\x12\\xe9\\xd2E'", 0.9996492266654968),
 ('http://www.wikidata.org/entity/Q7622276', 0.9996473789215088),
 ("b'\\xb7G#/\\x0f\\xbd\\x80~'", 0.9996356964111328),
 ("b'x\\x0bp\\xc6\\xd9\\x18\\x98\\x83'", 0.9996340274810791),
 ("b'g\\x80\\x81\\xe6\\x0b_\\xb1g'", 0.9996260404586792),
 ("b'\\xb0\\xebc\\xd9\\xc0QP<'", 0.9996208548545837),
 ("b'V>\\xb9C\\xad\\x80<\\xeb'", 0.9996188879013062)]

##Translation based Embedding with PyKEEN

In [1]:
#!pip install pykeen==1.3.0
!pip install pykeen --upgrade

In [2]:
import pykeen

In [3]:
pykeen.get_version()

'1.7.1-dev'

In [15]:
from pykeen.pipeline import pipeline
pipeline_result = pipeline(
    dataset='Nations',
    model='TransE',
    training_kwargs=dict(num_epochs=10)
)



INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.datasets:Loading cached preprocessed dataset from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/validation


Training epochs on cuda:0:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.


Evaluating on cuda:0:   0%|          | 0.00/201 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.06s seconds


In [6]:
model = pipeline_result.model

In [11]:
predicted_tails_df = model.get_tail_prediction_df( 'brazil', 'intergovorgs', triples_factory=pipeline_result.training
)

In [12]:
predicted_tails_df.head()

,tail_id,tail_label,score,in_training
0,0,brazil,-6.030367,False
2,2,china,-8.892767,False
11,11,uk,-9.659655,True
1,1,burma,-9.775414,False
9,9,netherlands,-9.880415,True


In [13]:
predicted_heads_df = model.get_head_prediction_df( 'conferences', 'brazil', triples_factory=pipeline_result.training)

In [14]:
predicted_heads_df.head()

,head_id,head_label,score,in_training
0,0,brazil,-6.119022,False
12,12,usa,-8.858175,True
4,4,egypt,-9.145296,False
5,5,india,-9.383718,True
11,11,uk,-9.411305,False


In [16]:
from pykeen.pipeline import pipeline
pipeline_result = pipeline(
    dataset='Nations',
    model='RotatE',
    training_kwargs=dict(num_epochs=10)
)


INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.datasets:Loading cached preprocessed dataset from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Loading from file:///root/.data/pykeen/datasets/nations/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/validation


Training epochs on cuda:0:   0%|          | 0/10 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/7 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.


Evaluating on cuda:0:   0%|          | 0.00/201 [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 0.06s seconds


In [17]:
model = pipeline_result.model

In [19]:
predicted_tails_df = model.get_tail_prediction_df( 'brazil', 'intergovorgs', triples_factory=pipeline_result.training)

In [20]:
predicted_tails_df.head()

,tail_id,tail_label,score,in_training
12,12,usa,-4.533686,True
11,11,uk,-4.747688,True
9,9,netherlands,-4.942078,True
10,10,poland,-4.980660,False
4,4,egypt,-5.039910,True
